In [4]:
import pandas as pd
from pathlib import Path
import requests
from IPython.display import display, Markdown
from config import column_names
from datetime import datetime


languages = ["jn", "ger", "en", "es", "it"
             ]

for language in languages:
    def load_markdown():
        url1 = "https://raw.githubusercontent.com/moritzvitt/moritzProjekt/markdown/prompts/_general_prompt.md"

        examples_url = f"https://raw.githubusercontent.com/moritzvitt/moritzProjekt/main/prompts/{language}_examples.md"
        # furigana_url = f"https://raw.githubusercontent.com/moritzvitt/moritzProjekt/markdown/prompts/{target_language}_add_furigana.md"
        url2 = examples_url
        # url3 = furigana_url
        # url2 = "https://raw.githubusercontent.com/moritzvitt/moritzProjekt/markdown/add_furigana.md"
        
        response1 = requests.get(url1)
        markdown1 = response1.text

        response2 = requests.get(url2)
        markdown2 = response2.text

        # response3 = requests.get(url3)
        # markdown3 = response3.text

        merged_markdown = markdown1 + "\n" + markdown2 + "\n" 
        # + markdown3
        return merged_markdown

    # load csv from test_dataframes 

    # load dataframe from csv
    df = pd.read_csv('../test_dataframes/japanese_items/items.csv', delimiter='\t', encoding='utf-8')
    # print(df.head())
    # print(df.shape)

    # load column_names from config.py
    df.columns = column_names

    df = df[[
            "Word", 
            "Context",
            "Context machine translation",
            "Context human translation",
            ]]

    current_time = datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d_%H-%M")
    print(formatted_time)

    # run 
    def run():
        markdown = load_markdown()
        # display(Markdown(markdown))
            # also safe the markdown to a new file
        # create final.md name with random number at the end
        # add the date in an f string



        with open(f"../complete_prompts/{language}_complete.md", "w") as file:
            file.write(markdown+'\n'+'\nThis is the table with the word sentence pairs: \n\n'+df.to_csv(sep='\t', encoding='utf-8', index=False))

    run()


2024-06-26_14-18
2024-06-26_14-18
2024-06-26_14-18
2024-06-26_14-18
2024-06-26_14-18


In [1]:
import genanki
from config import fields_config
import os
import time


def generate_anki_deck(df: pd.DataFrame) -> genanki.Package:
    """Generates an Anki deck from a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing card data.

    Returns:
        genanki.Package: The generated Anki package.
    """
    
    with open('templates/anki_card.html', 'r', encoding='utf-8') as content_file:
        content = content_file.read()

    # Splitting HTML content
    html_sections = content.split('<!-- html -->')

    # Assigning sections to qfmt, afmt, and css
    qfmt_html = html_sections[1]
    afmt_html = html_sections[2]

    with open('static/css/anki_card.css', 'r', encoding='utf-8') as content_file:
        css_code = content_file.read()

    # Ensure all columns are strings
    df = df.astype(str)

    # Define the Anki model
    model_id = 1607392319
    model = genanki.Model(
        model_id,
        'Language Learning with Netflix Model',
        fields = fields_config["fields"],
        templates=[
            {
                'name': 'Card 1',
                'qfmt': qfmt_html,
                'afmt': afmt_html,
            },
        ],
        css=css_code
    )

    # Create an Anki deck
    deck_id = model_id + 1  # Ensure deck_id is different from model_id
    deck = genanki.Deck(deck_id, 'lln_anki_deck')

    # Add cards to the deck
    for index, row in df.iterrows():
        my_note = genanki.Note(
            model=model,
            fields=[row['ID'], row['cloze'], row['hint'], row['definition'], row['notes'], row['image'], row['audio']],
        )
        deck.add_note(my_note)

    apkg_package = genanki.Package(deck)
    return apkg_package

@log_io
def export_df(df: pd.DataFrame, package: genanki.Package, native_language: str, output_file_path: str, encoding: str = 'utf-8') -> Tuple[str, str]:
    """Exports an Anki package and a cleaned DataFrame to CSV.

    Args:
        df (pd.DataFrame): The DataFrame to export.
        package (genanki.Package): The Anki package to save.
        native_language (str): The native language of the data.
        output_file_path (str): The path to save the files.
        encoding (str, optional): The encoding for the CSV file. Defaults to 'utf-8'.

    Returns:
        Tuple[str, str]: A tuple containing the paths to the exported Anki package and CSV file.
    """
    current_time = time.strftime("%Y%m%d%H%M%S", time.localtime())
    package_path = os.path.join(output_file_path, f'{native_language}_LLN_{current_time}.apkg')
    package.write_to_file(package_path)

    csv_file_path = os.path.join(output_file_path, f'{native_language}_LLN_{current_time}.csv')
    df.to_csv(csv_file_path, index=False, sep='\t', encoding=encoding)

    return package_path, csv_file_path


NameError: name 'pd' is not defined